In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from specim_test.specim.specfuncs import spec2d
from astropy.io import fits as pf

In [2]:
file_loc = '../galaxy_spectra/LRIS_data/LRIS_red_data_201213/keck_lris_red_A/Science/*'

In [6]:
file_list = []
out_file = []
for i,p in enumerate(glob.glob(file_loc)):
    if 'spec2d' in p and '0053m1' in p:
        file_list.append(p)
        frame = p.split('_')[-3][:5]
        out_file.append('J0053_LRIS_spectra/J0053_r'+ frame + '_test_skysub.fits')

In [4]:
file_list

['../galaxy_spectra/LRIS_data/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0026a-0053m1_LRISr_2020Dec13T045626.045.fits',
 '../galaxy_spectra/LRIS_data/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0026b-0053m1_LRISr_2020Dec13T045626.045.fits',
 '../galaxy_spectra/LRIS_data/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0028-0053m1_LRISr_2020Dec13T071213.306.fits',
 '../galaxy_spectra/LRIS_data/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0027-0053m1_LRISr_2020Dec13T062831.930.fits']

In [4]:
### This part is to create sky subtracted spectra. After that we can use those spectra for further analysis.
for i, p in enumerate(file_list):
    spec = spec2d.Spec2d(p, hext=12, xtrim=[360, 405])
    spec.set_dispaxis('y')
    spec.subtract_sky_2d()  #outfile=out_file[i], 
    #spec.szap(outfile=out_file[i], use_skymod=True)
    spec.szap_new(infile='0053m1_lens_spectra/0053m1_lens_median_image.fits')
    
    filled_data = spec.data
    mask = spec.szap_mask
    #print(filled_data.shape)
    wav_im = pf.open(p)[19].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    sky_im = pf.open(p)[14].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    ivar_im = pf.open(p)[13].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    
    ivar_im[mask] = 1.e-5
    #print(ivar_im.shape)
    sci_hdr = pf.Header()
    sci_hdr.append(('OBJECT', 'J0053-2012'))
    sci_hdr.append(('IMTYPE', 'SCIENCE_IMAGE'))
    sci_hdr.append(('', 'Trimmed and cleaned data by Pritom'))
    sci_im_hdu = pf.PrimaryHDU(data=filled_data, header=sci_hdr)
    
    ivar_hdr = pf.Header()
    ivar_hdr.append(('IMTYPE', 'IVAR_IMAGE'))
    ivar_im_hdu = pf.ImageHDU(data=ivar_im, header=ivar_hdr)
    
    sky_hdr = pf.Header()
    sky_hdr.append(('IMTYPE', 'SKY_MODEL'))
    sky_im_hdu = pf.ImageHDU(data=sky_im, header=sky_hdr)
    
    wav_hdr = pf.Header()
    wav_hdr.append(('IMTYPE', 'WAV_IMAGE'))
    wav_im_hdu = pf.ImageHDU(data=wav_im, header=wav_hdr)
    
    mask_hdr = pf.Header()
    mask_hdr.append(('IMTYPE', 'MASK'))
    cmt = 'pixels masked as cosmic rays and filled with pixel values at same pixel'\
        'location from median image'
    mask_hdr.append(('', cmt))
    mask_hdu = pf.ImageHDU(data=mask, header=mask_hdr)
    
    hdul = pf.HDUList([sci_im_hdu, ivar_im_hdu, sky_im_hdu, wav_im_hdu, mask_hdu ])
    hdul.writeto(out_file[i])


Loading file ../galaxy_spectra/LRIS_data/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0026a-0053m1_LRISr_2020Dec13T045626.045.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/LRIS_data/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0026a-0053m1_LRISr_2020Dec13T045626.045.fits (HDU=12)
The input dataset was trimmed
 xrange: 360:406.  yrange: 0:4096
Final data dimensions (x y): 46 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 46


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



sky model will be generated from data

 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


From this point sky subtracted data will be used

From now cosmic ray rejected data will be used

Loading file ../galaxy_spectra/LRIS_data/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r20121

In [8]:
for i, p in enumerate(file_list):
    spec = spec2d.Spec2d(p, hext=12, xtrim=[360, 405])#, transpose=True)
    spec.set_dispaxis('y')
    spec.subtract_sky_2d(use_skymod=True, outfile=out_file[i])
    #spec.szap_new(infile='0053m1_lens_spectra/0053m1_lens_median_image.fits', outfile=out_file[i])
    #pf.PrimaryHDU(filled_data).writeto(out_file[i])


Loading file ../galaxy_spectra/LRIS_data/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0026a-0053m1_LRISr_2020Dec13T045626.045.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/LRIS_data/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0026a-0053m1_LRISr_2020Dec13T045626.045.fits (HDU=12)
The input dataset was trimmed
 xrange: 360:406.  yrange: 0:4096
Final data dimensions (x y): 46 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 46


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



pypeit generated sky model will be used for sky subtraction

 Wrote sky subtracted data to J0053_LRIS_spectra/J0053_r0026a_test_skysub.fits

From this point sky subtracted data will be used
 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


Loading file ../galaxy_spectra/LRIS_data/LRIS_

In [ ]:
def szap(sp_1):

    sp_2 = pf.open('0053m1_lens_median_image.fits')[0].data
    mask = np.zeros(sp_1.shape, dtype=bool)
    #print(sp_1.shape)

    for k in range(0, sp_1.shape[0], 400):
        for i in range(0, sp_1.shape[1], 5):
            sp1 = sp_1[k:k+400, i:i+5]
            sp2 = sp_2[k:k+400, i:i+5]
            count1, bins1 = np.histogram(sp1.flatten(), bins=1000)
            count2, bins2 = np.histogram(sp2.flatten(), bins=1000)
            sm1 = 0
            sm2 = 0
            cnt1 = 0
            cnt2 = 0
            tot_pix =sp1.flatten().shape[0]

            for j,p in enumerate(count1):
                if p > cnt1:
                    cnt1 = p
                    max_bin1 = bins1[j+1]
                sm1 += p
                if sm1/tot_pix >= 0.8 :
                    border1 = 0.5*(bins1[j+1] + bins1[j])
                    break
            #print(border)
            for j,p in enumerate(count2):
                if p > cnt2:
                    cnt2 = p
                    max_bin2 = 0.5*(bins2[j+1] + bins2[j])
                sm2 += p
                if sm2/tot_pix >= 0.8 :
                    border2 = bins2[j+1]
                    break

            a = np.where(sp1>border1)
            a =np.transpose(a)

            if abs(border1 - max_bin1) < 50 :
                mp = 50    #/ abs(border1 - max_bin1)
            elif abs(border1 - max_bin1) < 100 :
                mp = 150
            elif abs(border1 - max_bin1) < 300:
                mp = 200
            else:
                mp = 400

            for j, p in enumerate(a):

                #print(sp2[p[0]][p[1]])
                if sp2[p[0]][p[1]] >0 :
                    #pass 
                    if sp1[p[0]][p[1]] > (sp2[p[0]][p[1]]+mp) :
                        mask[k+p[0]][i+p[1]] =1 
                else:
                    if sp1[p[0]][p[1]] > 3*border1 :
                        mask[k+p[0]][i+p[1]] =1
                        
    #print(mask)
    sp_1[mask] = sp_2[mask]
    
    mf =  (sp_1 > 200) | (sp_1 < -100)
    
    for k, p in enumerate(np.transpose(np.where(mf==True))):
        mask[p[0]][p[1]] = 1
        reg = sp_1[p[0]-10: p[0]+10, p[1]:p[1]+1]
        sp_1[p[0]][p[1]] = np.median(reg)
                
        #d[mask] = fill[mask] 
        #mm = m.astype('bool') | mask
    
    return sp_1, mask.astype(int)

In [9]:
## median image
spec_list = []
for i, p in enumerate(file_list):
    spec = spec2d.Spec2d(p, hext=12, xtrim=[360, 405])
    spec.set_dispaxis('y')
    spec.subtract_sky_2d()
    spec_list.append(spec.data)
median_im = np.median(np.asarray(spec_list), axis=0)
pf.PrimaryHDU(median_im).writeto('0053m1_lens_median_image.fits')


Loading file ../galaxy_spectra/LRIS_data/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0026a-0053m1_LRISr_2020Dec13T045626.045.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/LRIS_data/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0026a-0053m1_LRISr_2020Dec13T045626.045.fits (HDU=12)
The input dataset was trimmed
 xrange: 360:406.  yrange: 0:4096
Final data dimensions (x y): 46 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 46


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



sky model will be generated from data

 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


From this point sky subtracted data will be used

Loading file ../galaxy_spectra/LRIS_data/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0026b-0053m1_LRISr_2020Dec13T045626.045.fits
-

In [ ]:
spec.szap() #outfile='0053m1_lens_szap.fits'

In [ ]:
spec.do_waverect(doplot=True)

In [ ]:
profile = spec.spatial_profile(pixrange=[1600, 3500])

In [ ]:
spec.find_and_trace()

In [ ]:
spec.display_spec()

In [ ]:
spec.data[0]

In [ ]:
y2d, x2d = np.indices(spec.data.shape)

In [ ]:
y2d

In [ ]:
x2d.sum(axis=1)

In [ ]:
y2d1, x2d1 = np.indices(spec.data.T.shape)

In [ ]:
y2d1.sum(axis=0)

In [ ]:
x2d1

In [ ]:
a = y2d1.sum(axis=0)
a.repeat(51).reshape(4096, 51)

In [ ]:
a1 = np.array([1, 3, 5])
a1.repeat(2).reshape(3,2)

In [ ]:
from astropy.modeling import models
g = models.Gaussian1D(amplitude=[1, 2, 3], mean=[0, 0, 1],
                      stddev=[0.1, 0.2, 0.3], n_models=3)

In [ ]:
p = []

In [ ]:
p.append(g)

In [ ]:
p.append(g)

In [ ]:
p[0].mean.value

In [5]:
print(1.e-5)

1e-05


In [8]:
ivar_im.shape

(4096, 46)

In [6]:
np.sum(ivar_im==1.e-5)

92